<a href="https://colab.research.google.com/github/thilankam/GraphNeuralNetwork_PyG/blob/main/GCN_Model_with_Node_sets_U_and_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 47.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 19.6 MB/s eta 0:00:00


In [2]:
import torch_geometric
from torch_geometric.data import Data

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Define the number of nodes in U and V
num_nodes_u = 8
num_nodes_v = 235

# Create node feature matrices for U and V (here, we'll just use a random tensor)
u_features = torch.randn(num_nodes_u, 32)
v_features = torch.randn(num_nodes_v, 32)

# Create edge indices (assuming no edges for now)
edge_index = torch.tensor([], dtype=torch.long)

# Create the PyG Data object
data = Data(x=torch.cat([u_features, v_features], dim=0),
            edge_index=edge_index)

# Assign a node type to each node
data.u_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.u_mask[:num_nodes_u] = True
data.v_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.v_mask[num_nodes_u:] = True


Here, we define the number of nodes in U and V, create random node feature matrices for U and V, and create an empty edge index tensor. We then create a PyG Data object, which consists of node features and edge indices, and assign a node type to each node (in this case, U or V).

In [ ]:
# Split the edges into training and testing sets
train_edges, val_edges, test_edges = train_test_split_edges(data.edge_index, val_ratio=0.05, test_ratio=0.1)


We use the train_test_split_edges function to split the edge indices into training, validation, and testing sets.

In [ ]:
# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


We define a simple two-layer Graph Convolutional Network (GCN) using the GCNConv module from the PyG library.

In [ ]:
# Initialize the GCN model
model = GCN(in_channels=32, hidden_channels=16, out_channels=1)

# Define the loss function (binary cross-entropy)
criterion = torch.nn.BCEWithLogitsLoss()

# Define the optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    train_pos_edge_index = train_edges.to(device)
    train_pos_y = torch.ones(train_pos_edge_index.size(1), dtype=torch.float32, device=device)
    train_neg_edge_index = negative_sampling(train_pos_edge_index, num_nodes=data.num_nodes)
    train_neg_y = torch.zeros(train_neg_edge_index.size(1), dtype=torch.float32, device=device)
    train_edge_index = torch.cat([train_pos_edge_index, train_neg_edge_index], dim=-1)
    train_y = torch.cat([train_pos_y, train_neg_y], dim=0)
    out = model(data.x, train_edge_index).view(-1)
    loss = criterion(out, train_y)
    loss
